In [ ]:
from datetime import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import UFC_Scrape_Library as LIB

In [3]:
events_path = 'datasets/MMA_Events.csv'
decisions_path = 'datasets/MMA_Decisions.csv'
scorecards_path = 'datasets/MMA_Scorecards.csv'

# Scrape Events

In [4]:
urls_yearly_events = []
currentyear = datetime.now().year

for year in range(1994, currentyear + 1):
    newurl = 'https://mmadecisions.com/decisions-by-event/' + str(year) + '/'
    urls_yearly_events.append(newurl)

urls_yearly_events

['https://mmadecisions.com/decisions-by-event/1994/',
 'https://mmadecisions.com/decisions-by-event/1995/',
 'https://mmadecisions.com/decisions-by-event/1996/',
 'https://mmadecisions.com/decisions-by-event/1997/',
 'https://mmadecisions.com/decisions-by-event/1998/',
 'https://mmadecisions.com/decisions-by-event/1999/',
 'https://mmadecisions.com/decisions-by-event/2000/',
 'https://mmadecisions.com/decisions-by-event/2001/',
 'https://mmadecisions.com/decisions-by-event/2002/',
 'https://mmadecisions.com/decisions-by-event/2003/',
 'https://mmadecisions.com/decisions-by-event/2004/',
 'https://mmadecisions.com/decisions-by-event/2005/',
 'https://mmadecisions.com/decisions-by-event/2006/',
 'https://mmadecisions.com/decisions-by-event/2007/',
 'https://mmadecisions.com/decisions-by-event/2008/',
 'https://mmadecisions.com/decisions-by-event/2009/',
 'https://mmadecisions.com/decisions-by-event/2010/',
 'https://mmadecisions.com/decisions-by-event/2011/',
 'https://mmadecisions.com/d

In [5]:
df_event = pd.read_csv(events_path)
urls_events = df_event['url'].tolist()
    
for url in urls_yearly_events:

    if url in urls_events:
        continue

    df_event_new = LIB.scrape_event_data(url)

    df_event_all = pd.concat([df_event, df_event_new], ignore_index=True)

# Sort descending by date
df_event_all['Date'] = pd.to_datetime(df_event_all['Date']).dt.date
df_event_all = df_event_all.sort_values(by='Date', ascending=False)

df_event_all.head(10)

,Date,Event,NumFights,url
1516,2025-12-06,CW 199: Cage Warriors 199,2,https://mmadecisions.com/event/1590/CW-199-Cag...
1517,2025-12-06,UFC 323: Dvalishvili vs. Yan 2,5,https://mmadecisions.com/event/1591/UFC-323-Dv...
1518,2025-11-22,CW 198: Cage Warriors 198,8,https://mmadecisions.com/event/1589/CW-198-Cag...
1519,2025-11-22,UFC Fight Night 265: Tsarukyan vs. Hooker,5,https://mmadecisions.com/event/1588/UFC-Fight-...
1520,2025-11-21,CW 197: Cage Warriors 197,4,https://mmadecisions.com/event/1587/CW-197-Cag...
1521,2025-11-15,CW 196: Cage Warriors 196,3,https://mmadecisions.com/event/1586/CW-196-Cag...
1522,2025-11-15,UFC 322: Della Maddalena vs. Makhachev,7,https://mmadecisions.com/event/1585/UFC-322-De...
1523,2025-11-08,UFC on ESPN+ 122: Bonfim vs. Brown,4,https://mmadecisions.com/event/1584/UFC-on-ESP...
1524,2025-11-01,UFC on ESPN+ 121: Garcia vs. Onama,5,https://mmadecisions.com/event/1583/UFC-on-ESP...
1525,2025-10-25,UFC 321: Aspinall vs. Gane,9,https://mmadecisions.com/event/1582/UFC-321-As...


In [6]:
# Save to MMA_Events csv
#df_event_all.to_csv(events_path, index=False)

# Scrape Decisions

In [7]:
all_fight_data = []
df_decision = pd.read_csv(decisions_path)
urls_decisions = df_decision['url'].tolist()

for url in df_event_all['url']:

  if url in urls_decisions:
      continue

  try:
      df_decision_new = LIB.scrape_fight_data(url)
      df_decision = pd.concat([df_decision, df_decision_new], ignore_index=True)
  except Exception as e:
    continue
    
# Sort descending by date
df_decision = df_decision.sort_values(by='url', ascending=False)

df_decision.head(10)


,Event,Location,Venue,url
0,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9997/Mason-J...
1,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9997/Mason-J...
17415,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9997/Mason-J...
2,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9994/Kris-Ed...
3,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9994/Kris-Ed...
17416,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9994/Kris-Ed...
4,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9993/Josh-Re...
5,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9993/Josh-Re...
17417,CW 104: Shore vs. Malone,"Cardiff, Wales",Viola Arena,https://mmadecisions.com/decision/9993/Josh-Re...
6,Bellator 220: MacDonald vs. Fitch,"San Jose, California, USA",SAP Center,https://mmadecisions.com/decision/9990/Benson-...


In [8]:
# Save to MMA_Scorecards csv
#df_decision.to_csv(decisions_path, index=False)

# Scrape Scorecards

In [9]:
error_urls = []
df_scorecards = pd.read_csv(scorecards_path)
urls_scorecards = df_decision['url'].tolist()

for url in df_decision['url']:

  if url in urls_scorecards:
      continue

  try:
      df_scorecards_new = LIB.scrape_scorecard(url)
      df_scorecards = pd.concat([df_scorecards, df_scorecards_new], ignore_index=True)
  except Exception:
    error_urls.append(url)
    continue
    
# Sort descending by date
df_scorecards = df_scorecards.sort_values(by='url', ascending=False)

df_scorecards.head(10)


,Fight,Judge,Round,FighterOne,FighterTwo,ScoreOne,ScoreTwo,url
0,Mason Jones vs Donovan Desmae,Ben Cartlidge,1,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
2,Mason Jones vs Donovan Desmae,Mark Collett,1,Mason Jones,Donovan Desmae,9,10,https://mmadecisions.com/decision/9997/Mason-J...
3,Mason Jones vs Donovan Desmae,Ben Cartlidge,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
4,Mason Jones vs Donovan Desmae,David Lethaby,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
5,Mason Jones vs Donovan Desmae,Ben Cartlidge,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
6,Mason Jones vs Donovan Desmae,Mark Collett,3,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
7,Mason Jones vs Donovan Desmae,Mark Collett,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
8,Mason Jones vs Donovan Desmae,David Lethaby,2,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
1,Mason Jones vs Donovan Desmae,David Lethaby,1,Mason Jones,Donovan Desmae,10,9,https://mmadecisions.com/decision/9997/Mason-J...
13,Kris Edwards vs Josh Abraham,Mark Collett,2,Kris Edwards,Josh Abraham,9,10,https://mmadecisions.com/decision/9994/Kris-Ed...


In [ ]:
print("Error URLs:", error_urls)

In [10]:
# Save to MMA_Scorecards csv
#df_scorecards.to_csv(scorecards_path, index=False)